In [12]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime
# import hvplot.pandas
# import matplotlib.pyplot as plt
# from sklearn import svm
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from pandas.tseries.offsets import DateOffset
# from sklearn.metrics import classification_report

from utils.alpaca import alpaca_symbol_data
from utils.talltowide import TalltoWide

# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
# print(sp500Tickers)

In [13]:
# Sets up lists to hold the tickers
count = 0
tickers_0 = []
tickers_1 = []
tickers_2 = []
tickers_3 = []
tickers_4 = []
tickers_5 = []


size = len(sp500Tickers)
print(size)

# Reads the tickers from the Wiki Table into the ticker holders
for each_ticker in sp500Tickers:
    if count <= 99:
        tickers_0.append(each_ticker)
    if count >= 100 and count <= 199:
        tickers_1.append(each_ticker)
    if count >= 200 and count <= 299:
        tickers_2.append(each_ticker)
    if count >= 300 and count <= 399:
        tickers_3.append(each_ticker)
    if count >= 400 and count <= 499:
        tickers_4.append(each_ticker)
    if count >= 500:
        tickers_5.append(each_ticker)
    count = count + 1
# print(count)

# Adds SPY to the last list, which will be used for y in the machine learning models
tickers_5.append('SPY')


# Verify's appropriate lengths, should be 100 each except the last which should be 5
print(len(tickers_0))
# print(tickers_0)
print(len(tickers_1))
# print(tickers_1)
print(len(tickers_2))
# print(tickers_2)
print(len(tickers_3))
# print(tickers_3)
print(len(tickers_4))
# print(tickers_4)
print(len(tickers_5))
# print(tickers_5)


504
100
100
100
100
100
5


In [14]:
# Calls the API on each ticker

# All the tickers
tickerList =[tickers_0, tickers_1, tickers_2, tickers_3, tickers_4, tickers_5]


# Set start datetimes n years from today
years_back = 1

# Get the symbol_data and store it into a variable ohlcv_df
# ohlcv_df = alpaca_symbol_data(tickers, years_back)
ohlcv_df = pd.DataFrame()

# Iterates through every ticker and stores it in a DF, then saves that DF into the DB
StockCount = 100
for tickers in tickerList:
    ohlcv_df = alpaca_symbol_data(tickers, years_back)
    display(ohlcv_df.head())
    ohlcv_df.to_sql('stocks_'+ str(StockCount), engine, if_exists='replace')
    StockCount +=100


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-19 04:00:00+00:00,21.98,21.985,21.37,21.55,26643764,128347,21.641655,AAL
2021-04-20 04:00:00+00:00,21.01,21.020,19.96,20.36,47740892,236257,20.326026,AAL
2021-04-21 04:00:00+00:00,20.00,21.030,19.64,21.01,48033754,191826,20.593699,AAL
2021-04-22 04:00:00+00:00,21.57,21.610,20.04,20.07,72567588,290574,20.655093,AAL
2021-04-23 04:00:00+00:00,20.27,21.200,20.18,21.11,48357876,181554,20.858045,AAL


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-19 04:00:00+00:00,37.72,37.92,37.410,37.85,3086344,23425,37.783903,CAG
2021-04-20 04:00:00+00:00,37.73,38.35,37.505,38.27,3551909,29557,38.108433,CAG
2021-04-21 04:00:00+00:00,38.39,38.65,38.180,38.55,3130521,28887,38.461660,CAG
2021-04-22 04:00:00+00:00,38.59,38.69,38.045,38.17,2074361,20811,38.203525,CAG
2021-04-23 04:00:00+00:00,38.01,38.07,37.340,37.61,2685201,23639,37.602605,CAG


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-19 04:00:00+00:00,136.27,136.610,134.840,136.26,616095,10589,135.989666,AJG
2021-04-20 04:00:00+00:00,136.47,137.340,135.920,136.26,562944,10473,136.433863,AJG
2021-04-21 04:00:00+00:00,136.57,137.295,135.830,137.17,437274,9160,136.892308,AJG
2021-04-22 04:00:00+00:00,137.00,137.160,135.655,136.25,405983,9803,136.402039,AJG
2021-04-23 04:00:00+00:00,136.50,138.300,136.270,137.94,562948,9725,137.718109,AJG


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-19 04:00:00+00:00,130.60,131.09,129.47,131.09,866758,13218,130.423612,DGX
2021-04-20 04:00:00+00:00,130.50,131.40,128.17,129.37,1024356,14789,129.369187,DGX
2021-04-21 04:00:00+00:00,129.47,130.67,129.22,130.02,1166288,19732,129.797679,DGX
2021-04-22 04:00:00+00:00,131.11,133.90,128.33,133.69,2127915,25094,132.186699,DGX
2021-04-23 04:00:00+00:00,131.10,132.78,130.01,131.21,1586034,20479,131.328296,DGX


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-19 04:00:00+00:00,230.83,232.1953,228.87,230.50,3837150,71649,230.191173,CRM
2021-04-20 04:00:00+00:00,230.00,231.4500,226.89,227.96,3624123,69632,228.541365,CRM
2021-04-21 04:00:00+00:00,227.11,230.7900,226.67,230.56,4013916,63844,229.415149,CRM
2021-04-22 04:00:00+00:00,230.00,233.8900,229.01,231.35,4963892,83672,231.737397,CRM
2021-04-23 04:00:00+00:00,232.94,235.4900,231.75,233.51,5475468,78352,234.304060,CRM


,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-19 04:00:00+00:00,416.26,416.7400,413.79,415.21,75527000,450182,415.352416,SPY
2021-04-20 04:00:00+00:00,413.93,415.0859,410.59,412.17,79104051,476871,412.677447,SPY
2021-04-21 04:00:00+00:00,411.55,416.2900,411.36,416.07,64672721,359135,414.531238,SPY
2021-04-22 04:00:00+00:00,415.91,416.7800,411.13,412.27,97276832,587727,413.942573,SPY
2021-04-23 04:00:00+00:00,412.82,418.2500,412.79,416.74,73241320,362355,415.667363,SPY


In [15]:
filepath = Path('./resources/ohlcv_df_df.csv') 
filepath.parent.mkdir(parents=True, exist_ok=True) 
ohlcv_df.to_csv(filepath)

In [16]:
# Reads all the stocks from the DB
Stocks100 = pd.read_sql_table('stocks_100', engine, parse_dates='timestamp')
Stocks200 = pd.read_sql_table('stocks_200', engine, parse_dates='timestamp')
Stocks300 = pd.read_sql_table('stocks_300', engine, parse_dates='timestamp')
Stocks400 = pd.read_sql_table('stocks_400', engine, parse_dates='timestamp')
Stocks500 = pd.read_sql_table('stocks_500', engine, parse_dates='timestamp')
Stocks600 = pd.read_sql_table('stocks_600', engine, parse_dates='timestamp')

In [17]:
Stocks = pd.concat([Stocks100, Stocks200, Stocks300, Stocks400, Stocks500, Stocks600], axis=0)
StocksWide = TalltoWide(Stocks)
StocksWide

C:\Users\dlpst\anaconda3\envs\dev\envs\dev\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\dlpst\anaconda3\envs\dev\envs\dev\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


symbol,timestamp,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,SPY Open,SPY Close
0,2021-04-19 04:00:00,-0.006549,-0.019563,-0.005424,0.010110,0.005928,-0.004167,0.003415,0.007524,-0.001951,...,-0.005634,0.000611,-0.006974,-0.005660,0.003548,-0.007537,-0.004802,0.003582,416.26,415.21
1,2021-04-20 04:00:00,-0.004060,-0.030938,-0.020657,-0.013926,0.008417,0.010059,-0.002388,-0.012020,0.009338,...,-0.016717,0.001989,-0.005638,0.007258,0.007419,-0.011536,-0.050245,0.004704,413.93,412.17
2,2021-04-21 04:00:00,0.007149,0.050500,0.027873,0.008614,0.009199,0.001322,0.032889,0.018019,0.003625,...,0.027523,0.011623,0.017189,-0.002632,0.014997,0.009901,0.030146,0.003361,411.55,416.07
3,2021-04-22 04:00:00,0.003731,-0.069541,0.006533,-0.006850,-0.005423,-0.004857,0.007363,0.005618,-0.002852,...,-0.011270,-0.016418,-0.008364,-0.003046,0.004531,-0.017499,-0.021140,-0.003771,415.91,412.27
4,2021-04-23 04:00:00,0.015680,0.041441,-0.006120,0.015652,0.015222,0.006536,0.015741,-0.002508,0.005341,...,0.005246,0.011305,0.007570,-0.000085,0.003389,0.011777,0.035489,0.009112,412.82,416.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2022-04-11 04:00:00,-0.014713,0.028485,-0.021131,-0.017894,-0.029543,-0.018407,-0.042728,-0.027859,-0.026272,...,-0.023524,0.026032,-0.009245,-0.005213,-0.017041,-0.005064,0.004221,-0.032517,444.11,439.92
249,2022-04-12 04:00:00,-0.047840,0.005282,-0.001260,-0.001905,-0.011813,0.008872,-0.011187,-0.011687,-0.009098,...,-0.004651,-0.000413,-0.014289,0.007479,-0.003776,-0.021101,-0.016490,-0.021862,443.02,438.29
250,2022-04-13 04:00:00,0.024190,0.060436,-0.005142,0.018225,-0.036667,0.002029,0.013956,0.016332,0.003854,...,0.000692,0.007440,0.023294,0.015922,0.015073,0.009472,0.023338,0.008139,437.96,443.31
251,2022-04-14 04:00:00,-0.015418,0.000000,-0.010498,-0.031296,0.028255,-0.012738,-0.000783,-0.019903,-0.022669,...,0.012216,0.005121,-0.031983,0.006878,-0.016854,-0.032727,-0.012400,-0.013691,443.44,437.79


In [18]:
# Saves the raw data into a DB
StocksWide.to_sql('StocksWide', engine, if_exists='replace')

# 3Y

In [19]:
# For 3Y
# Read from DB
Stocks3Y100 = pd.read_sql_table('stocks3Y_100', engine, parse_dates='timestamp')
Stocks3Y200 = pd.read_sql_table('stocks3Y_200', engine, parse_dates='timestamp')
Stocks3Y300 = pd.read_sql_table('stocks3Y_300', engine, parse_dates='timestamp')
Stocks3Y400 = pd.read_sql_table('stocks3Y_400', engine, parse_dates='timestamp')
Stocks3Y500 = pd.read_sql_table('stocks3Y_500', engine, parse_dates='timestamp')
Stocks3Y600 = pd.read_sql_table('stocks3Y_600', engine, parse_dates='timestamp')

In [20]:
Stocks3Y = pd.concat([Stocks3Y100, Stocks3Y200, Stocks3Y300, Stocks3Y400, Stocks3Y500, Stocks3Y600], axis=0)
StocksWide3Y = TalltoWide(Stocks3Y)
StocksWide3Y

C:\Users\dlpst\anaconda3\envs\dev\envs\dev\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\dlpst\anaconda3\envs\dev\envs\dev\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


symbol,timestamp,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,SPY Open,SPY Close
0,2019-04-15 04:00:00,-0.007407,-0.025974,0.004930,0.003526,0.001607,0.013989,0.003131,-0.007174,0.004989,...,-0.001980,-0.004469,-0.003893,0.000394,-0.004127,-0.008318,-0.015292,-0.005890,290.27,289.97
1,2019-04-16 04:00:00,-0.040460,0.002061,-0.014297,-0.000351,-0.011705,-0.012321,-0.029202,-0.023523,-0.004116,...,0.004702,-0.018890,0.003408,-0.003627,-0.026297,0.007138,0.009623,-0.028602,290.96,290.16
2,2019-04-17 04:00:00,-0.034805,-0.002032,-0.017449,0.019320,-0.026102,-0.043524,-0.060132,-0.041053,-0.004727,...,0.000123,-0.035792,-0.000966,-0.003730,-0.048872,-0.013509,-0.003092,-0.014385,291.41,289.45
3,2019-04-18 04:00:00,0.005810,-0.001743,-0.006570,0.002015,-0.005640,0.002075,0.016079,0.005714,-0.000615,...,-0.005516,0.014691,0.000000,0.015929,0.002841,0.003796,-0.021109,0.007631,290.12,290.02
4,2019-04-22 04:00:00,-0.004741,-0.018182,-0.025050,0.009227,0.007217,-0.018915,0.020454,0.019010,0.002082,...,0.012581,-0.002428,0.007968,0.001748,0.015841,-0.004319,-0.008070,0.014182,289.17,290.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,2022-04-08 04:00:00,-0.006190,-0.016598,0.037742,-0.009665,0.011329,0.011742,-0.002632,-0.001377,-0.010250,...,0.016624,0.000625,-0.003106,0.003790,-0.002386,-0.001513,0.003298,0.011628,448.04,447.57
754,2022-04-11 04:00:00,-0.014713,0.028485,-0.021131,-0.017894,-0.029543,-0.018407,-0.042728,-0.027859,-0.026272,...,-0.023524,0.026032,-0.009245,-0.005213,-0.017041,-0.005064,0.004221,-0.032517,444.11,439.92
755,2022-04-12 04:00:00,-0.047840,0.005282,-0.001260,-0.001905,-0.011813,0.008872,-0.011187,-0.011687,-0.009098,...,-0.004651,-0.000413,-0.014289,0.007479,-0.003776,-0.021101,-0.016490,-0.021862,443.02,438.29
756,2022-04-13 04:00:00,0.024190,0.060436,-0.005142,0.018225,-0.036667,0.002029,0.013956,0.016332,0.003854,...,0.000692,0.007440,0.023294,0.015922,0.015073,0.009472,0.023338,0.008139,437.96,443.31


In [21]:
StocksWide.to_sql('StocksWide3Y', engine, if_exists='replace')

# 5 Day Moving Average